# Capstone 2

## Polish Bankruptcy

## Pre-processing and Training

In the pre-processing step I will impute the missing values and from the data, drop redundent high correlation variables from the dataset, complete a stratified train test split and and investigate scaling the training data.  Because of the high amount of skewed data the possibility of transforming the data may need to be done after the initial models.  

## Import Statements

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from random import randint, choice
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## Load Data

In [2]:
#load bankruptcy data and column key data
bankruptcy_data = pd.read_csv('bankruptcy_data_comb.csv')
data_columns = pd.read_csv('column_key.csv')

#load high correlation dataframe
dataCorrhigh = pd.read_csv('dataCorrhigh.csv')

#display high correlation pairs dataframe
print(dataCorrhigh.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Feature1     111 non-null    object 
 1   Feature2     111 non-null    object 
 2   Correlation  111 non-null    float64
dtypes: float64(1), object(2)
memory usage: 2.7+ KB
None


## Impute missing data

Based of the distribution of data and the number of outliers, I have made the decsion to imput the missing values based on the median of each column.

In [3]:
#create new dataframe by imputing missing values with the median
bankruptcy_complete = bankruptcy_data.apply(lambda x: x.fillna(x.median()),axis=0)

#display complete dataframe
bankruptcy_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43405 entries, 0 to 43404
Data columns (total 66 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X01     43405 non-null  float64
 1   X02     43405 non-null  float64
 2   X03     43405 non-null  float64
 3   X04     43405 non-null  float64
 4   X05     43405 non-null  float64
 5   X06     43405 non-null  float64
 6   X07     43405 non-null  float64
 7   X08     43405 non-null  float64
 8   X09     43405 non-null  float64
 9   X10     43405 non-null  float64
 10  X11     43405 non-null  float64
 11  X12     43405 non-null  float64
 12  X13     43405 non-null  float64
 13  X14     43405 non-null  float64
 14  X15     43405 non-null  float64
 15  X16     43405 non-null  float64
 16  X17     43405 non-null  float64
 17  X18     43405 non-null  float64
 18  X19     43405 non-null  float64
 19  X20     43405 non-null  float64
 20  X21     43405 non-null  float64
 21  X22     43405 non-null  float64
 22

## Drop redundant high correlation  columns


In [4]:
# drop first row of correlation dataframe with correlation of 1.  Data is identical 
dataCorrhigh = dataCorrhigh.drop(dataCorrhigh.index[0]).reset_index(drop=True)

#create empty final high correlation dataframe
corr_high_final = []

# remove redundant feature X07
dataCorrhigh = dataCorrhigh.drop(dataCorrhigh[(dataCorrhigh['Feature1'] == 'X07') | (dataCorrhigh['Feature2'] == 'X07')].index,axis = 0 ,inplace = False)
dataCorrhigh = dataCorrhigh.reset_index(drop=True)

# iterate over high correlation dataframe and use random number to determine final feture dataframe
for x in range(len(dataCorrhigh)):
    k = random.randint(0, 1)
    if k == 0:
        corr_high_final.append(dataCorrhigh['Feature1'][x])
        
    else:
        corr_high_final.append(dataCorrhigh['Feature2'][x])
   
print(corr_high_final)

['X56', 'X04', 'X58', 'X20', 'X08', 'X19', 'X31', 'X44', 'X10', 'X54', 'X31', 'X03', 'X50', 'X04', 'X14', 'X09', 'X56', 'X20', 'X43', 'X26', 'X44', 'X44', 'X44', 'X33', 'X22', 'X16', 'X22', 'X12', 'X44', 'X49', 'X20', 'X49', 'X58', 'X11', 'X34', 'X30', 'X52', 'X33', 'X47', 'X63', 'X36', 'X03', 'X51', 'X62', 'X54', 'X53', 'X30', 'X43', 'X35', 'X11', 'X36', 'X63', 'X35', 'X49', 'X09', 'X11', 'X11', 'X56', 'X58', 'X30', 'X63', 'X35', 'X11', 'X22', 'X33', 'X35', 'X33', 'X18', 'X44', 'X62', 'X33', 'X48', 'X62', 'X62', 'X58', 'X48', 'X10', 'X38', 'X06', 'X34', 'X34', 'X14', 'X35', 'X24', 'X24', 'X06', 'X06', 'X48', 'X24', 'X14', 'X34', 'X24', 'X56', 'X13', 'X13', 'X34', 'X22', 'X35', 'X13', 'X63', 'X44', 'X13', 'X63']


## Create final feature array 

In [5]:
# get unique feature array
feature_df = np.unique(corr_high_final)
print(feature_df)

['X03' 'X04' 'X06' 'X08' 'X09' 'X10' 'X11' 'X12' 'X13' 'X14' 'X16' 'X18'
 'X19' 'X20' 'X22' 'X24' 'X26' 'X30' 'X31' 'X33' 'X34' 'X35' 'X36' 'X38'
 'X43' 'X44' 'X47' 'X48' 'X49' 'X50' 'X51' 'X52' 'X53' 'X54' 'X56' 'X58'
 'X62' 'X63']


## Remove non correlated columns from bankruptcy data

In [6]:
#create new dataframe with high correlation features
bankruptcy_data_final = bankruptcy_complete[feature_df].copy()

# add back in the year and class features
bankruptcy_data_final = pd.concat([bankruptcy_data_final,bankruptcy_complete['Year']], axis = 1)
bankruptcy_data_final = pd.concat([bankruptcy_data_final,bankruptcy_complete['Class']], axis = 1)
print(bankruptcy_data_final.head(20))

         X03      X04       X06       X08      X09      X10       X11  \
0   0.396410   2.0472  0.388250   1.33050  1.13890  0.50494  0.249760   
1   0.472250   1.9447  0.000000   0.99601  1.69960  0.49788  0.261140   
2   0.267130   1.5548  0.000000   0.43695  1.30900  0.30408  0.312580   
3   0.458790   2.4928  0.149880   1.86610  1.05710  0.57353  0.092704   
4   0.229600   1.4063  0.187320   0.63070  1.15590  0.38677  0.187320   
5   0.359690   1.7502  0.000000   1.00830  1.97860  0.50206  0.286450   
6   0.289710   1.4705  0.000000   0.54454  1.73480  0.35256  0.125750   
7   0.705540  53.9540  0.000000  35.95700  0.65273  0.97294  0.693940   
8   0.053735   1.1263  0.000000   0.58223  1.33320  0.36798  0.043162   
9   0.142040   1.1694  0.000000   0.19279  2.11560  0.16163  0.184540   
10  0.188350   1.4400 -0.931900   1.25450  4.74470  0.55645  0.242270   
11  0.119890   2.0754 -0.084883   7.67410  0.90732  0.85551 -0.024300   
12  0.611470   3.0243  0.559830   1.85770  1.12680 

## Train test split


Train test split will be done with the stardard 75/25 split and the data stratified by bankruptcy class to maintain ratio of bankrupt and non-bankrupt entries.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(bankruptcy_data_final.drop(columns='Class'), bankruptcy_data_final.Class,
                                                    test_size=.25  ,random_state=5, stratify=bankruptcy_data_final.Class)

print(X_train)

           X03      X04       X06      X08      X09      X10       X11  \
12113 -0.83283  0.39990 -1.630900 -0.31339  0.88906 -0.43594 -1.068200   
3289   0.27021  2.05400  0.035034  2.84740  0.98773  0.72997  0.014925   
16567 -0.12099  0.72904  0.188430  0.74268  1.00990  0.40316  0.013167   
21367  0.37752  2.66550  0.297050  0.78826  1.12700  0.43244  0.119670   
32394  0.60189  6.70350  0.390000  8.44050  1.16000  0.89073  0.152710   
...        ...      ...       ...      ...      ...      ...       ...   
24299 -0.69448  0.25470 -0.072920 -0.53611  1.14410 -1.15570 -1.080300   
41350  0.28017  1.64960  0.217130  1.25060  1.07930  0.53940  0.143070   
38703  0.17282  1.91770  0.000000  0.81292  1.18680  0.44840  0.137730   
12352  0.49805  5.54020 -0.000078  5.64980  0.99346  0.84962  0.152910   
24962 -0.24260  0.54536 -0.054472  0.44388  6.12300  0.30742  0.051654   

            X12       X13       X14  ...      X50      X51      X52      X53  \
12113 -0.769670 -0.104620 -1.06

In [8]:
print(y_train)

12113    0
3289     0
16567    0
21367    0
32394    0
        ..
24299    0
41350    0
38703    0
12352    0
24962    0
Name: Class, Length: 32553, dtype: int64


In [9]:
print(X_test)

            X03      X04       X06       X08      X09       X10       X11  \
25162 -0.037208  0.82964  0.036874  1.170500  0.99219  0.407140  0.005390   
15362  0.381770  2.64830  0.160060  3.317500  1.95710  0.768380  0.294360   
4236   0.253630  2.00330  0.393040  2.890600  1.07750  0.730760  0.108000   
33893 -0.029308  0.96085 -0.376400  0.335800  1.21670  0.251390  0.062945   
40333  0.088177  1.38210  0.000000  3.188200  1.73650  0.761230  0.011931   
...         ...      ...       ...       ...      ...       ...       ...   
10939 -0.205130  0.58910  0.117250  0.241780  1.23530  0.168120  0.031665   
9520  -0.577590  0.39683 -0.132020  0.034792  1.38560  0.033622  0.100260   
41847  0.243250  1.62230  0.004878  1.023400  0.99943  0.477800  0.006726   
38614 -0.078133  0.73275 -0.010568  0.841350  1.00960  0.433760 -0.013002   
33968  0.169880  1.75800 -0.013478  1.093600  0.99949  0.370360  0.019724   

            X12       X13       X14  ...      X50      X51       X52      X

In [10]:
print(y_test)

25162    0
15362    0
4236     0
33893    0
40333    0
        ..
10939    0
9520     0
41847    0
38614    0
33968    0
Name: Class, Length: 10852, dtype: int64


In [11]:
#display X training and test sizes
X_train.shape, X_test.shape

((32553, 39), (10852, 39))

In [12]:
# display y training and test sizes
y_train.shape, y_test.shape

((32553,), (10852,))

In [13]:
# verify proper training columns
print(X_train.columns)

Index(['X03', 'X04', 'X06', 'X08', 'X09', 'X10', 'X11', 'X12', 'X13', 'X14',
       'X16', 'X18', 'X19', 'X20', 'X22', 'X24', 'X26', 'X30', 'X31', 'X33',
       'X34', 'X35', 'X36', 'X38', 'X43', 'X44', 'X47', 'X48', 'X49', 'X50',
       'X51', 'X52', 'X53', 'X54', 'X56', 'X58', 'X62', 'X63', 'Year'],
      dtype='object')


## Scale the data

Based on the range of the data and the degree of outliers the data was scaled before the logistic regression.

In [14]:
# scale fit and transform the data using standard scaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [15]:
print (X_train_scaled)

            X03       X04       X06       X08       X09       X10       X11  \
12113 -0.163551 -0.019446 -0.213395 -0.025616 -0.037721 -0.073549 -0.339471   
3289   0.028093 -0.014577  0.010597 -0.019986 -0.035235  0.009176 -0.032724   
16567 -0.039875 -0.018477  0.031222 -0.023735 -0.034676 -0.014012 -0.033221   
21367  0.046737 -0.012777  0.045826 -0.023654 -0.031725 -0.011934 -0.003059   
32394  0.085720 -0.000891  0.058324 -0.010024 -0.030893  0.020583  0.006298   
...         ...       ...       ...       ...       ...       ...       ...   
24299 -0.139514 -0.019874 -0.003918 -0.026012 -0.031294 -0.124618 -0.342898   
41350  0.029824 -0.015768  0.035081 -0.022830 -0.032927 -0.004345  0.003568   
38703  0.011172 -0.014978  0.005887 -0.023610 -0.030218 -0.010802  0.002055   
12352  0.067679 -0.004315  0.005876 -0.014995 -0.035090  0.017666  0.006355   
24962 -0.061004 -0.019018 -0.001437 -0.024267  0.094182 -0.020805 -0.022322   

            X12       X13       X14  ...       X50 

## Create the Logistic Regression object 

In [16]:
# create the regression object
clf = LogisticRegression(max_iter=1000)
# fit the scaled and training data
clf.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [17]:
# run the regression model
y_te_pred = clf.predict(X_test_scaled)
y_predict_training = clf.predict(X_train_scaled)

## Accuracy Scores

In [18]:
# print test accuracy
print("\n")
print("[Test] Accuracy score: (y_test, y_te_pred)",accuracy_score(y_test, y_te_pred))

# print training accuracy
print("\n")
print("[Training] Accuracy score: (y_train, y_predict_training)",accuracy_score(y_train, y_predict_training))



[Test] Accuracy score: (y_test, y_te_pred) 0.9517139697751567


[Training] Accuracy score: (y_train, y_predict_training) 0.9518631155346665


## Classification Report

In [19]:
# print classification scores  0 is non bankrupt entities 1 is bankrupt entitiies
print("[Training Classification Report]")
print(classification_report(y_train, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_te_pred))

[Training Classification Report]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     30985
           1       0.52      0.01      0.01      1568

    accuracy                           0.95     32553
   macro avg       0.74      0.50      0.49     32553
weighted avg       0.93      0.95      0.93     32553

[Test Classification Report]
              precision    recall  f1-score   support

           0       0.95      1.00      0.98     10329
           1       0.44      0.01      0.02       523

    accuracy                           0.95     10852
   macro avg       0.70      0.50      0.50     10852
weighted avg       0.93      0.95      0.93     10852



The overall accuracy score is .95 due to the the size of the non bankrupt entities. The logistic regression model predicts the non bankrupt entities very well but the does not do as well with the banrkrupt entities at .54 for the training data and fares worse on the test data at .44.

## Save train test split


In [20]:
#save test and train dataframes to file
X_train_scaled.to_csv('X_train_scaled.csv', index = True)
X_test_scaled.to_csv('X_test_scaled.csv', index = True)
y_train.to_csv('y_train.csv', index = True)
y_test.to_csv('y_test.csv', index = True)

## Summary and further steps


In the pre-processing step, I imputed the missing values, removed redundant correlation pairs and created the final feature dataframe. Then I created the train test split, scaled the data and trained the logistic regression model. Finally I checked the accuracy scores and classification report. The logistic regression model predicted the non bankrupt entities with 95% accuracy but only 54% for the training data and slightly worst for the test data at 44%. In the modeling step I will explore other models and apply resampling to evaluate the results. Lastly, I evaluate performance metrics and identify the best performing model.